# **Fully Connected Neural Network: A `CUDA` and `C++` Implementation**

## **Prepare workspace**

In [16]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/ParaProgram/Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ParaProgram/Project


## **Edit `Makefile`**

In [17]:
%%writefile Makefile

# Compilers
CXX := g++
CXX_FLAGS := -std=c++17 -ggdb
NVCC := nvcc

# Folders
BIN := bin
SRC := src
INCLUDE := include

EXECUTABLE := nn_main

all: $(BIN)/$(EXECUTABLE)

run: clean all
	clear
	./$(BIN)/$(EXECUTABLE)

$(BIN)/$(EXECUTABLE): $(SRC)/*.cu $(SRC)/*.cpp
	$(NVCC) -I $(INCLUDE) $^ -o $@

clean:
	-rm $(BIN)/*

Overwriting Makefile


## **Compile**

In [18]:
!make

nvcc -I include src/main.cu src/nn.cu src/utils_device.cu src/utils_host.cpp -o bin/nn_main


## **Run**

In [ ]:
# Extract data from `.gz`
# Only need to run once!
!pip install patool
import patoolib
patoolib.extract_archive("fashion/t10k-images-idx3-ubyte.gz" ,outdir="fashion")
patoolib.extract_archive("fashion/t10k-labels-idx1-ubyte.gz" ,outdir="fashion")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 3.5 MB/s eta 0:00:00


INFO patool: Extracting fashion/t10k-images-idx3-ubyte.gz ...
INFO:patool:Extracting fashion/t10k-images-idx3-ubyte.gz ...
INFO patool: running /usr/bin/7z e -ofashion -- fashion/t10k-images-idx3-ubyte.gz
INFO:patool:running /usr/bin/7z e -ofashion -- fashion/t10k-images-idx3-ubyte.gz
INFO patool: ... fashion/t10k-images-idx3-ubyte.gz extracted to `fashion'.
INFO:patool:... fashion/t10k-images-idx3-ubyte.gz extracted to `fashion'.
INFO patool: Extracting fashion/t10k-labels-idx1-ubyte.gz ...
INFO:patool:Extracting fashion/t10k-labels-idx1-ubyte.gz ...
INFO patool: running /usr/bin/7z e -ofashion -- fashion/t10k-labels-idx1-ubyte.gz
INFO:patool:running /usr/bin/7z e -ofashion -- fashion/t10k-labels-idx1-ubyte.gz
INFO patool: ... fashion/t10k-labels-idx1-ubyte.gz extracted to `fashion'.
INFO:patool:... fashion/t10k-labels-idx1-ubyte.gz extracted to `fashion'.


'fashion'

In [19]:
# Run the program
!./bin/nn_main

Train Images: 10000 with size 784 each.
Train Labels: 10000 labels loaded.
First train image pixel value: 0
First train label: 9
